In [5]:
import torch.nn as nn
import torch

In [6]:
# inputs and targets
# inputs (temp, rainfall, humidity)(obervations x each variable-temp,rain.humidity)
# targets (apples, oranges) (outputs per observation x outputs-apples,oranges)

inputs = torch.tensor([[73, 67, 43],
                       [91, 88, 64],
                       [87, 134, 58],
                       [102, 43, 37],
                       [69, 96, 70], 
                       [73, 67, 43],
                       [91, 88, 64],
                       [87, 134, 58],
                       [102, 43, 37],
                       [69, 96, 70],
                       [73, 67, 43],
                       [91, 88, 64],
                       [87, 134, 58],
                       [102, 43, 37],
                       [69, 96, 70]], dtype = torch.float32)

targets = torch.tensor([[56, 70],
                        [81, 101],
                        [119, 133],
                        [22, 37],
                        [103, 119],
                        [56, 70],
                        [81, 101],
                        [119, 133],
                        [22, 37],
                        [103, 119],
                        [56, 70],
                        [81, 101],
                        [119, 133],
                        [22, 37],
                        [103, 119]], dtype = torch.float32)


In [8]:
# creating a TensorDataset which allows access to rows as tuples
# creating a DataLoader to slipt data into batches for training.
# DataLoader also provides utilities such as shuffling and sampling

from torch.utils.data import TensorDataset, DataLoader

In [9]:
# defining training dataset

train_ds = TensorDataset(inputs, targets)

In [25]:
# defining DataLoader to define batch size etc during training
# train_dl contains 5 observations and corressponding target tuples 
# DataLoader combines a dataset and a sampler. Provides iterators over the dataset
# shuffle shuffles the data in the dataset before each epoch
# next(iter()) returns the next batch of obervations and corressponding targets from train_ds

batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle = True)
next(iter(train_dl))

[tensor([[ 69.,  96.,  70.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 87., 134.,  58.],
         [ 69.,  96.,  70.]]), tensor([[103., 119.],
         [119., 133.],
         [ 22.,  37.],
         [119., 133.],
         [103., 119.]])]

In [17]:
# we can use nn.linear instead of initializing weights and biases manually y=wx+b and w,b are randomly initialized

model = nn.Linear(3, 2) # 3 factors and 2 outputs
print(model.weight)
print(model.weight.size())
print(model.bias)
print(model.bias.size())



Parameter containing:
tensor([[-0.5101, -0.4320, -0.4786],
        [-0.3628,  0.1757, -0.4952]], requires_grad=True)
torch.Size([2, 3])
Parameter containing:
tensor([-0.0137, -0.1904], requires_grad=True)
torch.Size([2])


In [18]:
# Instead of manually updating weights and biases optimizers(can choose) can be used

opt = torch.optim.SGD(model.parameters(), lr = 1e-5) # lr is learning rate to update w,b after calculating loss and gradients


In [20]:
# Loss function. Mean Squared Error calculation is embedded in mse_loss function. Can choose other loss calculations too

import torch.nn.functional as F
loss_fn = F.mse_loss
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(27471.2383, grad_fn=<MseLossBackward>)


In [26]:
# Training model
# using a function called fit() to train the model over a specific number of epochs
# it needs as inputs the number of epochs, model, loss function and optimiser
# defining the fit() function

def fit(num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xb, yb, in train_dl: # train_dl is 1 batch
            # Generating/Sampling predictions
            pred = model(xb)# xb obervation of the three factors
            loss = loss_fn(pred, yb)# yb corressponding output-yeilds of apples and oranges
            # Gradient descent
            loss.backward()
            opt.step() # calling opt to update weights and biases
            opt.zero_grad() # making the gradients zero after the iteration as PyTorch accumulates gradients
        print("Training Loss: ", loss_fn(model(inputs), targets))
            

In [27]:
# training the model for 100 epochs

fit(100, model, loss_fn, opt)

Training Loss:  tensor(8670.6465, grad_fn=<MseLossBackward>)
Training Loss:  tensor(2865.5603, grad_fn=<MseLossBackward>)
Training Loss:  tensor(1108.7699, grad_fn=<MseLossBackward>)
Training Loss:  tensor(551.1914, grad_fn=<MseLossBackward>)
Training Loss:  tensor(383.3872, grad_fn=<MseLossBackward>)
Training Loss:  tensor(320.0350, grad_fn=<MseLossBackward>)
Training Loss:  tensor(295.6017, grad_fn=<MseLossBackward>)
Training Loss:  tensor(281.0715, grad_fn=<MseLossBackward>)
Training Loss:  tensor(271.5856, grad_fn=<MseLossBackward>)
Training Loss:  tensor(262.4278, grad_fn=<MseLossBackward>)
Training Loss:  tensor(254.6761, grad_fn=<MseLossBackward>)
Training Loss:  tensor(247.4583, grad_fn=<MseLossBackward>)
Training Loss:  tensor(240.1487, grad_fn=<MseLossBackward>)
Training Loss:  tensor(233.0982, grad_fn=<MseLossBackward>)
Training Loss:  tensor(225.9633, grad_fn=<MseLossBackward>)
Training Loss:  tensor(219.1916, grad_fn=<MseLossBackward>)
Training Loss:  tensor(212.7139, grad

In [ ]:
# making predictions after training. Passing the same input dataset as testset

